In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
package_dir = "../input/pretrained-models.pytorch-master/"
sys.path.insert(0, package_dir)
import pretrainedmodels
package_dir = '../input/early-stopping-pytorch'
sys.path.append(package_dir)
from pytorchtools import EarlyStopping
import numpy as np
import pandas as pd
import scipy as sp
from functools import partial
from sklearn import metrics
from sklearn.model_selection import KFold
from collections import Counter, OrderedDict
import json
import time
import cv2
import albumentations
from albumentations import torch as AT
import gc
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
from sklearn.metrics import cohen_kappa_score

# To have reproducible results and compare them
seedValue = 2019
np.random.seed(seedValue)
torch.manual_seed(seedValue)
os.environ['PYTHONHASHSEED'] = str(seedValue)
torch.cuda.manual_seed(seedValue)
torch.cuda.manual_seed_all(seedValue) 
torch.backends.cudnn.deterministic = True  
torch.backends.cudnn.benchmark = False

# Specify GPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5" 
device_ids = [0,1]
device = torch.device("cuda:0")

def quadratic_kappa(y_hat, y, coef):
    for pred in enumerate(y_hat):
            if pred[1] < coef[0]:
                y_hat[1] = 0
            elif pred[1] >= coef[0] and pred[1] < coef[1]:
                y_hat[1] = 1
            elif pred[1] >= coef[1] and pred[1] < coef[2]:
                y_hat[1] = 2
            elif pred[1] >= coef[2] and pred[1] < coef[3]:
                y_hat[1] = 3
            else:
                y_hat[1] = 4
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device=device)

# pre-processing

In [2]:
def crop_image1(img,tol=7):
    # 'tol' is tolerance
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img 
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        
        return img

In [3]:
transform = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.HorizontalFlip(),
    albumentations.RandomBrightness(),
    albumentations.JpegCompression(80),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])
transform_valid_test = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.JpegCompression(80),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])


In [4]:
# dfのimage_idから対応する画像を読み込んだり、前処理をかけたりする関数
class RetinopathyDataset(Dataset):
    def __init__(self, csv_file, transform, datatype='train'):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.datatype = datatype

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.datatype=='train':
            img_name = os.path.join('../input/aptos2019-blindness-detection/train_images',
                                self.data.loc[idx, 'id_code'] + '.png')
        elif self.datatype=='train_old':
            img_name = os.path.join('../input/diabetic-retinopathy-resized/resized_train',
                                self.data.loc[idx, 'image'] + '.jpeg')
        else:
            img_name =  os.path.join('../input/aptos2019-blindness-detection/test_images',
                                     self.data.loc[idx, 'id_code'] + '.png')   
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.circle_crop(img, sigmaX=10)
        img = self.transform(image=img)
        img = img['image']
        if self.datatype=='train':
            label = torch.tensor(self.data.loc[idx, 'diagnosis'])
            return {'image': img,
                    'labels': label
                    }
        elif self.datatype=='train_old':
            label = torch.tensor(self.data.loc[idx, 'level'])
            return {'image': img,
                    'labels': label
                    }
        else:
            return {'image': img}
        
    def circle_crop(self, img, sigmaX=10):   
        height, width, depth = img.shape    

        x = int(width/2)
        y = int(height/2)
        r = np.amin((x,y))

        circle_img = np.zeros((height, width), np.uint8)
        cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
        img = cv2.bitwise_and(img, img, mask=circle_img)
        img = crop_image_from_gray(img)
        img = cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
        return img 
    

## kappa_lossによる最適化をするclass．未使用．使い方勉強中
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [5]:
def load_model(pretrained='imagenet'):
    class Net(nn.Module):
      def __init__(self, model):
        super(Net, self).__init__()
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.last_linear = nn.Sequential(
                              nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                              nn.Dropout(p=0.25),
                              nn.Linear(in_features=2048, out_features=2048, bias=True),
                              nn.ReLU(),
                              nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                              nn.Dropout(p=0.5),
                              nn.Linear(in_features=2048, out_features=1, bias=True),
                             )
        self.l1 = nn.Sequential(*list(model.children())[:-1]).to(device)
        self.last = list(model.children())[-1]

      def forward(self, x):
        x = self.l1(x)
        x = x.view(x.size()[0], -1)
        x = self.last(x)
        return x
    
    model = Net(pretrainedmodels.__dict__['resnet101'](pretrained=pretrained))
    
    return model

# train

In [6]:
class train(object):
    def __init__(self, params):
        self.lr = params.get('lr')
        self.img_size = params.get('img_size')
        self.batch_size = params.get('batch_size')
        self.n_epochs = params.get('n_epochs')
        self.n_freeze = params.get('n_freeze')
        self.coef = params.get('coef')
        self.criterion = params.get('criterion')
        self.num_workers = params.get('num_workers')
        self.load_state = params.get('load_state')
        self.load_path = params.get('load_path')
        self.save_path = params.get('save_path')
        self.device = params.get('device')
        self.n_folds = params.get('n_folds')
        self.use_valid = False
        self.early_stop = params.get("early_stop")
        self.patience = params.get("patience")
    
    def get_train(self,data):
        self.train_data = data
        
    def get_valid(self,data):
        self.valid_data = data
        self.use_valid = True
        
    def fit(self, use_cv=False, train_idx=None, valid_idx=None):
        since = time.time()
        # Model
        if self.load_state:
            model = load_model(pretrained=None)
            # load params
            if torch.cuda.device_count() > 1:
                state_dict = torch.load(self.load_path)
                new_state_dict = OrderedDict()
                for k, v in state_dict.items():
                    name = k[7:] # remove `module.`
                    new_state_dict[name] = v
                model.load_state_dict(new_state_dict)
            else:    
                model.load_state_dict(torch.load(self.load_path, map_location=self.device))
        else:
            model = load_model(pretrained='imagenet')
        if torch.cuda.device_count() > 1: 
            model = nn.DataParallel(model, device_ids).cuda()
        model = model.to(device)

        plist = [{'params': model.parameters(), 'lr': self.lr}]
        optimizer = optim.Adam(plist, lr=self.lr)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=10)
        
        if use_cv:
            train_sampler = SubsetRandomSampler(train_idx)
            valid_sampler = SubsetRandomSampler(valid_idx)
            data_loader_train = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers,sampler=train_sampler)
            data_loader_valid = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers,sampler=valid_sampler)
        elif self.use_valid:
            data_loader_train = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers)
            data_loader_valid = torch.utils.data.DataLoader(self.valid_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers)
        else:
            data_loader_train = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers)
            
        if self.early_stop:
            early_stopping = EarlyStopping(patience=self.patience, verbose=True)
        
        for epoch in range(self.n_epochs):
            if epoch == self.n_freeze:      
                for param in model.parameters():
                    param.requires_grad = True

            print('Epoch {}/{}'.format(epoch, self.n_epochs - 1))
            print('-' * 10)
            scheduler.step()
            model.train()
            running_loss = 0.0
            kappa = 0
            steps = 0
            with tqdm(data_loader_train, total=int(len(data_loader_train))) as pbar:
                for bi, d in enumerate(pbar):
                    inputs = d["image"]
                    labels = d["labels"].view(-1, 1)
                    inputs = inputs.to(self.device, dtype=torch.float)
                    labels = labels.to(self.device, dtype=torch.float)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(True):
                        outputs = model(inputs)
                        loss = self.criterion(outputs, labels)
                        # <- insert custom mse
                        loss.backward()
                        optimizer.step()

                    running_loss += loss.mean().item() 
                    y_hat = torch.Tensor.cpu(outputs.view(-1)).detach()
                    y = torch.Tensor.cpu(labels.view(-1)).detach()
                    kappa += quadratic_kappa(y_hat, y, self.coef).mean().item()
                    steps += 1
                    pbar.set_postfix(OrderedDict(mse_loss = running_loss / steps,
                                                 kappa_score = kappa / steps))
                    
            epoch_loss = running_loss / steps
            print('Training Loss: {:.4f}'.format(epoch_loss))
            kappa = kappa / steps
            print('Training Kappa: {:.4f}'.format(kappa))
            
            if self.use_valid or use_cv:
                model.eval()
                kappa = 0
                steps = 0
                
                with tqdm(data_loader_valid , total=int(len(data_loader_valid))) as pbar:
                    for step, batch in enumerate(pbar):
                        inputs = batch["image"]
                        labels = batch["labels"].view(-1, 1)
                        inputs = inputs.to(self.device, dtype=torch.float)
                        labels = labels.to(self.device, dtype=torch.float)

                        with torch.no_grad():
                            outputs = model(inputs)

                        y_hat = torch.Tensor.cpu(outputs.view(-1))
                        y = torch.Tensor.cpu(labels.view(-1))
                        kappa += quadratic_kappa(y_hat, y, self.coef).mean().item()
                        steps += 1
                        pbar.set_postfix(OrderedDict(kappa_score = kappa / steps))
                    
                kappa = kappa / steps
                print('Validation Kappa: {:.4f}'.format(kappa))

                if self.early_stop:
                    eval_loss = 1 - kappa
                    early_stopping(eval_loss, model)
                    if early_stopping.early_stop:
                        print("Early stopping")
                        break

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
        if not use_cv:
            torch.save(model.state_dict(), self.save_path)

        del(model, data_loader_train)
        gc.collect()
        torch.cuda.empty_cache() 
        
    def cv(self):
        num_train = len(self.train_data)
        indices = list(range(num_train))
        kf = KFold(n_splits=self.n_folds, random_state=1337, shuffle=True)

        train_idx = []
        valid_idx = []

        for t, v in kf.split(indices):
            train_idx.append(t)
            valid_idx.append(v)

        # Training                        
        for fold in np.arange(self.n_folds):
            print('Fold:',fold)
            self.fit(use_cv=True,train_idx=train_idx[fold],valid_idx=valid_idx[fold])

In [7]:
#params
params = {
    "lr" : 1e-5,
    "img_size" : 224,
    "batch_size" : 64,
    "n_epochs" : 20,
    "n_freeze" : 1,
    "num_workers" : 8,
    "coef" : [0.5, 1.5, 2.5, 3.5],
    "criterion" : nn.MSELoss(),
    "load_state" : False,
    "load_path" : None, 
    'save_path' : None,
    "device" : device,
    "n_folds" : 2,
    "early_stop" : True,
    "patience" : 3,
}

#training for the lazy, like me

# cv: new
if 0:
    Mytrain = train(params)
    Mytrain.get_train(RetinopathyDataset(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                            transform=transform, datatype='train'))
    Mytrain.cv()

# train:old & valid:new
if 0:
    
    params['save_path'] =  "../input/mmmodel/model_old_test.bin"
    Mytrain = train(params)
    Mytrain.get_train(RetinopathyDataset(csv_file="../input/diabetic-retinopathy-resized/trainLabels.csv",
                                            transform=transform, datatype='train_old'))
    Mytrain.get_valid(RetinopathyDataset(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                           transform=transform, datatype='train'))
    Mytrain.fit()
    
# train:new
if 1:
    params['n_epochs'] = 15
    params['load_state'] = True
    params['load_path'] =  "../input/mmmodel/model_old_test.bin" 
    params['save_path'] =  "../input/mmmodel/model_new_test.bin"        
    Mytrain = train(params)
    Mytrain.get_train(RetinopathyDataset(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                           transform=transform, datatype='train'))
    Mytrain.fit()

Epoch 0/14
----------



Training Loss: 1.1776
Training Kappa: 0.6177
Epoch 1/14
----------



Training Loss: 0.9974
Training Kappa: 0.6831
Epoch 2/14
----------



Training Loss: 0.9312
Training Kappa: 0.7022
Epoch 3/14
----------



Training Loss: 0.8688
Training Kappa: 0.7205
Epoch 4/14
----------


...


Training Loss: 0.6462
Training Kappa: 0.7793
Epoch 11/14
----------



Training Loss: 0.6472
Training Kappa: 0.7747
Epoch 12/14
----------



Training Loss: 0.6396
Training Kappa: 0.7815
Epoch 13/14
----------



Training Loss: 0.6411
Training Kappa: 0.7752
Epoch 14/14
----------



Training Loss: 0.6287
Training Kappa: 0.7827
Training complete in 32m 56s


# Inference

In [6]:
load_path = "../input/mmmodel/model_new_test.bin"

model = load_model(pretrained=None)
if torch.cuda.device_count() > 1:
    state_dict = torch.load(load_path)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
else:    
    model.load_state_dict(torch.load(load_path, map_location=self.device))

model = model.to(device)

model.eval()

Net(
  (l1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1

In [7]:
test_dataset = RetinopathyDataset(csv_file='../input/aptos2019-blindness-detection/sample_submission.csv',
                                      transform=transform_valid_test, datatype='test')

In [8]:
test_bs = 16
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_bs, shuffle=False, num_workers=4)
test_preds = np.zeros((len(test_dataset), 1))
for i, x_batch in tqdm(enumerate(tqdm(test_data_loader))):
    x_batch = x_batch["image"]
    pred = model(x_batch.to(device))
    test_preds[i * test_bs:(i + 1) * test_bs] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [9]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4


sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)

In [10]:
sample["diagnosis"].value_counts()

2    780
1    532
0    465
3    137
4     14
Name: diagnosis, dtype: int64